# Assignment - Week 3

(1) open ./Week_3/smi_list_from_Week_2, parse a list of smiles 


(2) open ./Week_3/Week_3_iupac_name_cas_no, convert iupac or cas number to a smiles string

If pubchempy returns multiple compounds, please use the 0th one. 

If pubchempy returns no compounds, please just skip it.


(3) Merge (1) and (2), remove duplicates

Please note that the canonical smiles from PubChem needs to be 're-canonicalized' in RDKit to remove duplicates


(4) Collect the molecules that consist of C or H or O or N (and no other elements) and not more than 20 nonhydrogen atoms (mol.GetNumHeavyAtoms()), classify them in terms of stoichiometry

The dictionary should look like:

{'C11H22O2': ['CCCCCCC(=O)OC(C)CC', 'CCCCCCC1COC(C)(C)O1', 'CCCCC(CCCC)C(=O)OC', 'CCCCC(CC)COC(=O)CC', 'CCCCCCCCCOC(C)=O', 'CCCCCCOC(=O)CCCC', 'CCCCCOC(=O)CCCCC', 'CCCCCCCOC(=O)CCC', 'CCCCC(=O)OC(C)CCCC', 'CCCCCCCC(=O)OCCC', 'CCCCCCC(C)(C)CC(=O)O', 'CCCCCCC(=O)OCCCC', 'CCCCCCC1(C)OCCCO1', 'CCCCCCCC(=O)OC(C)C', 'CC(C)CCCC(C)CCC(=O)O', 'CCCCCCC(=O)OC(C)(C)C', 'CCCCCCCCOC(=O)CC', 'CCCCCC(C)OC(=O)CCC', 'CCCCCCCCCCC(=O)O', 'CCCCCCCCC(C)C(=O)O', 'CCCCCCCCCCOC=O', 'CCCCCCCCC(=O)OCC', 'CCOC(=O)CCCCCC(C)C', 'CCOC(=O)CCCCC(C)CC', 'CCCCCCCCCC(=O)OC', 'CCCCCCCOC(=O)C(C)C'], 'H2O': ['O'], ... }

Avoid writing '1's. e.g.) H2O1 --> H2O

Follow alphabetical orders. e.g.) 'C8H10N4O2'

(5) Write ./Week_3/Assignment.json .


In [1]:
# If pubchempy returns no compounds, please just skip it. Perhaps try-except can be used?
# Maybe many molecules are skipped. I brought the molecules quite randomly including very tricky ones, 
# and PubChemPy is not always perfect.
a = []
print(a[0]) #IndexError

'''
try:
   # do something
except IndexError:
    continue # skip this cycle and proceed to the next cycle
'''

IndexError: list index out of range

In [2]:
import pubchempy as pcp
from rdkit import Chem
pubchem_smi = []
with open('./Week_3/Week_3_iupac_name_cas_no','r') as f:
    for a_line in f:
        name = a_line.split()[0]
        try:
            compound = pcp.get_compounds(name,'name')[0]
            pubchem_smi.append(compound.canonical_smiles)
        except IndexError:
            continue
        
week_2_smi = []
with open('./Week_3/smi_list_from_Week_2','r') as f:
    for a_line in f:
        mol = Chem.MolFromSmiles(a_line.split()[0])
        week_2_smi.append( Chem.MolToSmiles(mol) )

all_smi = list(set(pubchem_smi + week_2_smi))

RDKit WARNING: [08:27:19] WARNING: not removing hydrogen atom without neighbors


In [3]:
print(len(all_smi))

6908


In [4]:
from collections import Counter
import json
stoi_classified = {}
for smi in all_smi:
    mol = Chem.MolFromSmiles(smi)
    mol = Chem.AddHs(mol)
    atom_counter = Counter([atom.GetSymbol() for atom in mol.GetAtoms()])

    if mol.GetNumHeavyAtoms() < 20 and len(set(atom_counter.keys()) - set(['C', 'H', 'N', 'O'])) == 0:
        stoi = ''
        for key in sorted(atom_counter.keys()):
            if atom_counter[key] != 1:
                stoi += key + str(atom_counter[key])
            else:
                stoi += key

        try:
            stoi_classified[stoi].append(smi)
        except:
            stoi_classified[stoi] = [smi]

#print(stoi_classified)           
with open('./Week_3/Assignment.json', 'w') as f:
    json.dump(stoi_classified, f, indent = 4)

RDKit WARNING: [08:27:20] WARNING: not removing hydrogen atom without neighbors
